# HW2: Numpy cho Khoa học dữ liệu - Tiền xử lý dữ liệu

Quy trình tiền xử lý dữ liệu này được thể hiện thông qua các bước như sau: 
- Đọc và load dữ liệu
- Kiểm tra tính hợp lệ của dữ liệu 
- Xử lý missing values
- Tính toán thống kê mô tả, kiểm định giả thiết thống kê
- Feature engineering
- Xác định và loại bỏ (nếu cần thật sự cần thiết) đối với các giá trị ngoại lai
- Chuẩn hoá (Normalization) cho từng đặc trưng
- Điều chuẩn khoảng giá trị phù hợp dữ liệu Non-Gaussian Distribution
- Điều chuẩn dữ liệu (Standardization) hay Z-score để đạt trung bình 0 và phương sai 1 trước khi sử dụng thuật toán dựa trên gradient hoặc feature engineering bằng các kỹ thuật dimensional reduction

Trước tiên tiến hành import thư viện `Numpy` cho tiền xử lý, `sys` và `os` hỗ trợ việc đọc và load dữ liệu, các hàm xử lý các yêu cầu về tiền xử lý dữ liệu từ file `data_processing.py`

In [1]:
import sys
import os
import numpy as np

# Thêm src vào path để import được
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data_processing import (
    load_csv_numpy, fill_missing_categorical, ordinal_encode_experience, label_encode,
    remove_outliers_iqr, create_new_features, validate_data_integrity, min_max_normalize, 
    log_transformation, decimal_scaling, standard_scale, robust_scale, 
    calculate_t_test_ind, save_processed_csv, perform_pca
)

## Đọc và load dữ liệu

Từ các dữ liệu đã được chuẩn bị sẵn trong thư mục `data/raw`, tiến hành đọc và load dữ liệu

In [2]:
train_path = os.path.join('..', 'data', 'raw', 'aug_train.csv')
test_path = os.path.join('..', 'data', 'raw', 'aug_test.csv')

# Khai báo kiểu dữ liệu cho từng đặc trưng
types_train = [
    'i8',   # enrollee_id
    'U50',  # city
    'f8',   # city_development_index
    'U50',  # gender
    'U50',  # relevent_experience
    'U50',  # enrolled_university
    'U50',  # education_level
    'U50',  # major_discipline
    'U50',  # experience
    'U50',  # company_size
    'U50',  # company_type
    'U50',  # last_new_job
    'i8',   # training_hours
    'f8'    # target
]

# Bỏ đặc trưng cuối cho tập test
types_test = types_train[:-1]

header_train, data_train = load_csv_numpy(train_path, types_train)
header_test, data_test = load_csv_numpy(test_path, types_test)

## Kiểm tra tính hợp lệ của dữ liệu

Bước đầu của việc tiền xử lý dữ liệu là kiểm tra tính hợp lệ, ở đây có 2 điểm đáng chú ý cần được xem xét:
- Kiểm tra thuộc tính "training_hours" có dương hay không
- Kiểm tra thuộc tính "city_development_index" có nằm trong khoảng 0 - 1 hay không 

In [3]:
validate_data_integrity(data_train)

Dữ liệu sạch, không có giá trị vô lý


## Xử lý missing values

Ý tưởng xử lý missing values được chia ra 2 hướng chính: 
- Điền vào giá trị phổ biến nhất cho các vị trí thiếu
- Gán trực tiếp giá trị "Unknown" vào các vị trí thiếu

Ở đây, thuộc tính `gender` và `major_discipline` được áp dụng cách 2 vì việc một ứng viên không điền giới tính hoặc chuyên ngành có thể mang mục đích cá nhân (ví dụ như muốn giấu thông tin, hoặc học chuyên ngành lạ không có trong danh sách). Vì vậy, việc gán giá trị "Unknown" giúp nó trở thành một thông tin để  xây dựng mô hình tốt hơn, bởi lẽ nếu điền vào giá trị phổ biến nhất (ví dụ điền hết là "Male" chẳng hạn) thì vô tình sẽ làm nhiễu dữ liệu (dữ liệu mang tính biased lớn).

Các thuộc tính còn lại thường bị mất dữ liệu chủ yếu do yếu tố  ngẫu nhiên hoặc do trong quá trình thu thập dữ liệu người được khảo sát lười điền vào chứ không phải do có những lựa chọn khác ngoài lề. Bởi vậy, ưu tiên điền vào giá trị phổ biến nhất để  có thể duy trì phân phối tổng thể của dữ liệu, tránh tạo ra quá nhiều giá trị "Unknown" gây loãng dữ liệu.

In [4]:
def fill_missing_pipeline(data):
    data = fill_missing_categorical(data, 'gender', strategy='unknown') 
    data = fill_missing_categorical(data, 'major_discipline', strategy='unknown')
    
    cat_cols = ['enrolled_university', 'company_size', 'company_type', 
                'education_level', 'last_new_job']
    for col in cat_cols:
        data = fill_missing_categorical(data, col, strategy='mode')
        
    data = fill_missing_categorical(data, 'experience', strategy='mode')
    return data

In [5]:
data_train = fill_missing_pipeline(data_train)
data_test = fill_missing_pipeline(data_test)

## Kiểm định giả thuyết thống kê

Ở đây sẽ tập trung kiểm định T-test độc lập để so sánh giá trị trung bình của đặc trưng `training_hours` giữa hai nhóm ứng viên: nhóm không đổi việc (Target=0) và nhóm muốn đổi việc (Target=1) với mục đích là để trả lời cho câu hỏi: "Việc ứng viên tham gia đào tạo nhiều giờ hơn có liên quan đến việc họ muốn bỏ việc hay không?". Ý tưởng đặt ra như sau: 
- Đặt giả thiết (Hypothesis) cho câu hỏi:
  + Giả thiết H0 (Null Hypothesis): $\mu_0 = \mu_1$ (Thời gian đào tạo trung bình của hai nhóm là như nhau).
  + Giả thiết H1 (Alternative Hypothesis): $\mu_0 \neq \mu_1$ (Có sự khác biệt về thời gian đào tạo).
- Lấy ra 2 mẫu dữ liệu của 2 thuộc tính `training_hours` và `target`, tính phương sai mẫu cho 2 mẫu dữ liệu này và sử dụng loại kiểm định T-test cho 2 phương sai không bằng nhau, với công thức sai số chuẩn (Standard Error - SE) được dùng là: $SE = \sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}$.
- Ngưỡng đánh giá (Critical Value): Xét với độ tin cậy 95% (mức ý nghĩa $\alpha=0.05$), từ đó đưa ra kết luận. 

In [6]:
hours = data_train['training_hours']
target = data_train['target']

group_0 = hours[target == 0.0]
group_1 = hours[target == 1.0]

t_stat, mean0, mean1 = calculate_t_test_ind(group_0, group_1)

if abs(t_stat) > 1.96: # Mức ý nghĩa 5%
    print("Bác bỏ H0: Ứng viên tham gia đào tạo nhiều giờ hơn có liên quan đến việc họ muốn bỏ việc")
    if mean1 > mean0:
        print("Người học nhiều hơn có xu hướng nhảy việc cao hơn")
    elif mean1 < mean0:
        print("Người học ít hơn có xu hướng nhảy việc cao hơn")
else:
    print("Chấp nhận H0: Ứng viên tham gia đào tạo nhiều giờ hơn không liên quan đến việc họ muốn bỏ việc")

Bác bỏ H0: Ứng viên tham gia đào tạo nhiều giờ hơn có liên quan đến việc họ muốn bỏ việc
Người học ít hơn có xu hướng nhảy việc cao hơn


## Feature engineering

Đầu tiên là chuyển đổi dữ liệu dạng chuỗi sang dạng số, áp dụng cho các giá trị của thuộc tính `experience` để giải quyết dưới dạng chuỗi hỗn hợp (ví dụ như "1", "5", "<1", ">20"). Ý tưởng xử lý như sau: 
- `"<1"` $\rightarrow$ `0`: Quy ước chưa có kinh nghiệm là 0 năm.
- `">20"` $\rightarrow$ `21`: Quy ước trên 20 năm là 21 (để giữ tính thứ tự lớn nhất).
- `""` (Rỗng) $\rightarrow$ `0`: Xử lý missing value (giả định thiếu là chưa có kinh nghiệm).
- Với các giá trị còn lại (ví dụ "5", "10") thì ép kiểu sang float.

In [7]:
exp_train = ordinal_encode_experience(data_train)
exp_test = ordinal_encode_experience(data_test)

Tiếp theo là tạo ra 2 đặc trưng mới: 
- Đặc trưng 1: `Interaction` 
  + Công thức: `cdi_exp = cdi * exp_numeric`
  + Ý nghĩa: Kết hợp Chỉ số phát triển thành phố và Kinh nghiệm. Ví dụ như một người có 20 năm kinh nghiệm (Exp = 21) sống ở thành phố rất phát triển (CDI = 0.9) sẽ có chỉ số này rất cao ($\approx 18.9$). Điều này cho thấy người này thuộc nhóm chuyên gia cao cấp ổn định, hành vi nhảy việc sẽ khác với người mới ra trường ở vùng kém phát triển.
- Đặc trưng 2: `Training Intensity` 
  + Công thức: `intensity = th / (exp_numeric + 1.0)`
  + Ý nghĩa: Đo Cường độ học tập của ứng viên. Lấy ví dụ như người A học 100 giờ và có Kinh nghiệm 1 năm thì sẽ có Training Intensity cao hơn người B học 100 giờ và có kinh nghiệm 20 năm, cho thấy người A rất chăm chỉ học hỏi với kinh nghiệm còn ít ỏi của mình, còn người B thì mức độ học được đánh giá là bình thường so với thâm niên. Đặc trưng này giúp mô hình phân biệt được ai là người đang nỗ lực học để thăng tiến sự nghiệp nhanh chóng (thường sẽ có Training Intensity cao) so với những người chỉ học duy trì.

In [8]:
cdi_exp_train, intensity_train = create_new_features(data_train, exp_train)
cdi_exp_test, intensity_test = create_new_features(data_test, exp_test)

Lấy ra các đặc trưng gốc dạng số để xử lý bước tiếp theo

In [9]:
th_train = data_train['training_hours']
cdi_train = data_train['city_development_index']
th_test = data_test['training_hours']
cdi_test = data_test['city_development_index']

## Xử lý các giá trị ngoại lai

Trọng tâm của phần này là sử dụng phương pháp IQR (Interquartile Range), nhưng trước tiên cần xử lý các đặc trưng bị lệch (skewed) là `training_hours` và `intensity` bằng cách áp dụng kỹ thuật `log transformation`. Lý do là nếu dùng IQR ngay trên dữ liệu thô, giá trị bị xô lệch lớn sẽ bị coi là ngoại lai và bị xóa, bởi thế mà cần dùng `log transformation` để khoảng cách giữa các giá trị được thu hẹp lại và phân phối trở nên đối xứng hơn. Lúc này, IQR sẽ hoạt động chính xác hơn, chỉ loại bỏ những giá trị nhiễu thực sự và giữ lại những giá trị cao hợp lý.

Các kỹ thuật đáng chú ý cho log transformation nằm ở 2 chỗ:
- **Shift (Dịch chuyển)**: `if np.min(array) < 0: array = array - np.min(array)` Bước này đảm bảo dữ liệu luôn không âm trước khi đưa vào hàm Log (vì log của số âm là không xác định).
- **Log1p**: Sử dụng `np.log1p(array)` tương đương với $\ln(1 + x)$. Việc cộng thêm 1 giúp xử lý trường hợp $x=0$ (vì $\ln(0) = -\infty$, gây lỗi tính toán).

Tác dụng chính của `log transformation` là biến đổi phân phối có đặc điểm long tail thành dạng gần giống hình chuông (Gaussian) hơn. Điều này cực kỳ quan trọng vì các thuật toán máy học như Linear/Logistic Regression hoạt động tốt nhất trên phân phối chuẩn.

In [10]:
th_train_log = log_transformation(th_train)
th_test_log = log_transformation(th_test)

intensity_train_log = log_transformation(intensity_train)
intensity_test_log = log_transformation(intensity_test)

Ý tưởng chính cho việc xác định và loại bỏ các giá trị ngoại lại (hàm `remove_outliers_iqr`) như sau:
- Áp dụng phương pháp IQR:
  + Tính $Q1$ (25%) và $Q3$ (75%).
  + Tính $IQR = Q3 - Q1$.
  + Xác định biên: $[Q1 - 1.5 \times IQR, Q3 + 1.5 \times IQR]$. Bất cứ điểm dữ liệu nào nằm ngoài khoảng này được coi là ngoại lai.
- Hàm trả về một mảng Boolean các giá trị True/False (mảng mask). Mảng mask này chỉ tính toán trên đặc trưng `training_hours` và cũng chính là biến `mask_train`, sau đó áp dụng mask này cho các đặc trưng khác như `cdi_train`, `exp_train`, `intensity_train`... Mục đích ở đây là nếu gặp trường hợp ứng viên A có giá trị của đặc trưng `training_hours` bị ngoại lai, toàn bộ thông tin của ứng viên A (tất cả các giá trị ở dòng của ứng viên A) sẽ bị xoá để đảm bảo tính toàn vẹn của dữ liệu. Lí do cho việc xoá đi toàn bộ là bởi nếu chỉ xóa `training_hours` mà giữ lại `target`, dòng dữ liệu đó sẽ bị lệch pha và gây lỗi khi xây dựng mô hình.

Ở đây xử lý ngoại lai chỉ áp dụng cho tập dữ liệu train, không áp dụng cho tập dữ liệu test.

In [11]:
# Sử dụng IQR để lọc dựa trên training_hours
mask_train = remove_outliers_iqr(th_train_log)

# Áp dụng mask để lọc cho dữ liệu train (các đặc trưng thuộc loại định lượng)
th_train_log = th_train_log[mask_train]
cdi_train = cdi_train[mask_train]
exp_train = exp_train[mask_train]
cdi_exp_train = cdi_exp_train[mask_train]
intensity_train = intensity_train[mask_train]
y_train = data_train['target'][mask_train]

# Lọc dữ liệu train (các đặc trưng thuộc loại định tính) để đồng bộ
data_train_filtered = data_train[mask_train]

## Chuẩn hoá (Normalization) cho từng đặc trưng

Kỹ thuật `min_max_normalize` được áp dụng cho đặc trưng `city_development_index` với cơ chế hoạt động như sau: Co giãn dữ liệu về đúng một khoảng cố định, thường là trong đoạn [0, 1].

Công thức: $$X_{new} = \frac{X - X_{min}}{X_{max} - X_{min}}$$

Kỹ thuật này mang lại 2 lợi ích: 
- Các giá trị của đặc trưng `city_development_index` thông qua bộ dữ liệu cho thấy nó được giới hạn rõ ràng trong đoạn [0, 1]. Vì vậy nên áp dụng kỹ thuật này không phải lo việc bị đụng phải các giá trị ngoại lai vốn là hạn chế của `min_max_normalize`. 
- Kỹ thuật này giúp giữ nguyên khoảng cách tỷ lệ giữa các thành phố. Ví dụ như với thành phố A (0.8) và thành phố B (0.4), sau khi chuẩn hoá thì A vẫn gấp đôi B mà không làm xô lệch đi cấu trúc phân phối gốc của dữ liệu.

In [12]:
cdi_train_norm = min_max_normalize(cdi_train)
cdi_test_norm = min_max_normalize(cdi_test)

Kỹ thuật `decimal_scaling` được áp dụng cho đặc trưng `experience` với cơ chế hoạt động như sau: Di chuyển dấu thập phân của số liệu sang trái cho đến khi giá trị tuyệt đối nhỏ hơn 1.

Công thức: $$X_{new} = \frac{X}{10^j}$$ 
    (với $j$ là số nguyên nhỏ nhất để $|X_{new}| < 1$).

Kỹ thuật này mang lại lợi ích:
- Đặc thù của dữ liệu trong `experience` là số nguyên dương, thường nằm trong khoảng 0 đến 30 năm. Ở đây giá trị lớn nhất là khoảng hơn 20.
- Với $j=2$ (chia cho 100), 5 năm trở thành 0.05, 20 năm trở thành 0.20. Với dữ liệu đã chuẩn hoá (0.20) này, chúng ta khi nhìn vào vẫn có thể hiểu đó là 20 năm. Kỹ thuật này giúp giảm độ lớn số học để thu hẹp đi chênh lệch với giá trị của các đặc trưng khác để xây dựng mô hình hiệu quả mà vẫn giữ nguyên ý nghĩa ban đầu của dữ liệu.

In [13]:
exp_train_dec = decimal_scaling(exp_train)
exp_test_dec = decimal_scaling(exp_test)

## Điều chuẩn khoảng giá trị phù hợp dữ liệu Non-Gaussian Distribution

Sau khi dùng kỹ thuật `log transformation` thì kỹ thuật `Robust Scaling` cũng được áp dụng tiếp cho 2 đặc trưng `training_hours` và `intensity` với cơ chế hoạt động như sau: Sử dụng các đại lượng thống kê vị trí (Median, Quartile) thay vì đại lượng độ lớn (Mean, Std).

Công thức: $$X_{new} = \frac{X - Median}{IQR}$$ (với $IQR = Q3 - Q1$).

Kỹ thuật này mang lại 1 số lợi ích:
- `training_hours` thông thường có phân phối lệch (Non-Gaussian Distribution) và chứa ngoại lai, lấy ví dụ như đa số học 20h, nhưng lại có người học 300h. Nếu dùng kỹ thuật `standard_scale` để chuẩn hoá về độ lệch chuẩn thì có khả năng giá trị 300h sẽ kéo Mean lên cao và làm tăng độ lệch chuẩn (Std). Kết quả là các giá trị bình thường (20h) sẽ bị ép về một khoảng rất nhỏ quanh giá trị 0, làm mất thông tin chi tiết của đa số dữ liệu.
- Thay vì như vậy, dùng Median và IQR là đặc trưng của `Robust Scaling`. Kỹ thuật này giải quyết tốt vấn đề dữ liệu Non-Gaussian Distribution hoặc chứa nhiều giá trị bất thường bằng cách dùng giá trị Median làm trung tâm của dữ liệu. Giá trị ngoại lai 300h không ảnh hưởng đến Median, do đó mà khoảng cách giữa các giá trị bình thường (10h, 20h, 30h) được bảo toàn tốt hơn, không bị kéo theo các giá trị ngoại lai vốn là thiểu số trong tập dữ liệu. Đây cũng là kỹ thuật giúp tránh đi nhiễu trong dữ liệu tốt nhất.

In [14]:
th_train_robust = robust_scale(th_train_log)
th_test_robust = robust_scale(th_test_log)

intensity_train_robust = robust_scale(intensity_train)
intensity_test_robust = robust_scale(intensity_test)

## Điều chuẩn dữ liệu (Standardization) hay Z-score để đạt trung bình 0 và phương sai 1 trước khi sử dụng thuật toán dựa trên gradient hoặc feature engineering bằng các kỹ thuật dimensionality reduction

Đối với đặc trưng `cdi_exp` và các đặc trưng thuộc loại định tính, cần áp dụng kỹ thuật `standard_scale` để  điều chuẩn dữ liệu hay Z-score để đạt trung bình $\mu$ = 0 và phương sai $\sigma^2$ = 1 với cơ chế hoạt động đưa dữ liệu về phân phối chuẩn tắc.

Công thức như sau: $$X_{new} = \frac{X - \mu}{\sigma}$$

Có 1 số lý do để chọn kỹ thuật này cho các đặc trưng này:
- `cdi_exp` vốn dĩ là thuộc tính được tạo thêm từ tích của hai thuộc tính có sẵn, bởi thế nên phân phối nên có xu hướng quy về chuẩn tắc (theo định lý giới hạn trung tâm).
- Đối với các đặc trưng thuộc loại định tính, sau khi áp dụng `label encoding` để chuyển về dạng số phục vụ cho việc xây dựng mô hình thì nên điều chuẩn để có trung bình và phương sai đồng nhất, giúp cho thuật toán Gradient Descent khi áp dụng sẽ có xu hướng hội tụ nhanh và ổn định.

Bắt đầu điều chuẩn cho đặc trưng `cdi_exp` trước

In [15]:
cdi_exp_train_std = standard_scale(cdi_exp_train)
cdi_exp_test_std = standard_scale(cdi_exp_test)

Tổng hợp lại các đặc trưng đã xử lý chuẩn hoá, điều chuẩn từ trước

In [16]:
feature_list_train = [cdi_train_norm, exp_train_dec, th_train_robust, intensity_train_robust, cdi_exp_train_std]
feature_list_test = [cdi_test_norm, exp_test_dec, th_test_robust, intensity_test_robust, cdi_exp_test_std]

Tiến hành `label encoding` cho các đặc trưng thuộc loại định tính, sau đó tiến hành điều chuẩn các đặc trưng này

In [17]:
cat_cols = ['gender', 'relevent_experience', 'enrolled_university', 
            'education_level', 'major_discipline', 'company_size', 
            'company_type', 'last_new_job', 'city']

for col in cat_cols:
    enc_train = label_encode(data_train_filtered, col)
    feature_list_train.append(standard_scale(enc_train))
    
    enc_test = label_encode(data_test, col)
    feature_list_test.append(standard_scale(enc_test))

Sau đó gộp lại thành 1 dữ liệu hoàn chỉnh 

In [18]:
X_train_final = np.column_stack(feature_list_train)
X_test_final = np.column_stack(feature_list_test)

Bước cuối cùng trong quy trình tiền xử lý sẽ là **giảm chiều dữ liệu** (Dimensionality Reduction) bằng kỹ thuật `PCA` (Principal Components Analysis). Kỹ thuật này đi qua 5 bước xử lý như sau: 

- **Bước 1: Center data**
  
    + Công thức: $X_{centered} = X - \mu$
      
    + Ý nghĩa: `PCA` hoạt động dựa trên việc tối đa hóa phương sai. Để tính phương sai chính xác, dữ liệu cần được dời về điểm 0 (mean = 0 như đã được chuẩn hoá từ trước) bằng cách lấy giá trị của mỗi đặc trưng trừ đi giá trị trung bình.
 
- **Bước 2: Covariance Matrix (Ma trận hiệp phương sai)**

    + Công thức: $\Sigma = \frac{1}{N-1} X^T X$
  
    + Ý nghĩa: Đo lường mối quan hệ tuyến tính giữa các đặc trưng bằng cách sử dụng phép nhân ma trận `np.dot` giữa $X^T$ and $X$, sau đó chia kết quả này cho n_samples - 1 (sử dụng Bessel's correction) để tính phương sai mẫu (sample covariance), giúp cho kết quả không bị biased.
 
- **Bước 3: Eigen Decomposition (Phân rã trị riêng)**

    + Ý nghĩa: Tìm ra các hướng (eigenvectors) mà dữ liệu biến thiên nhiều nhất và độ lớn của sự biến thiên đó (eigenvalues) bằng cách dùng `np.linalg.eigh`.
- **Bước 4: Sắp xếp và Chọn k thành phần (k = n_components)**

    + Ý nghĩa: PCA quan tâm đến các thành phần có phương sai lớn nhất. Vậy nên ta sắp xếp và lấy ra `n_components` đặc trưng đầu tiên tương ứng với k hướng (eigenvectors) quan trọng nhất, chứa nhiều thông tin nhất (k giá trị eigenvalues lớn nhất).
 
- **Bước 5: Transform (Chuyển đổi)**

    + Công thức: $Y = X \cdot W$, trong đó $W$ là ma trận các eigenvector đã chọn ở bước 4. 
    + Ý nghĩa: Chuyển đổi dữ liệu gốc sang chiều mới (k chiều như đã lấy). Kích thước mới của dữ liệu là $(N \times D) \cdot (D \times K) \rightarrow (N \times K)$. Kết quả cho thấy dữ liệu đã giảm chiều.

In [19]:
X_train_pca, explained_var = perform_pca(X_train_final, n_components=X_train_final.shape[1])

Dữ liệu sau khi xử lý thì tiến hành lưu vào thư mục `data/processed` 

In [20]:
processed_train_path = os.path.join('..', 'data', 'processed', 'aug_train.csv')
processed_test_path = os.path.join('..', 'data', 'processed', 'aug_test.csv')

save_processed_csv(X_train_pca, y_train, processed_train_path)
save_processed_csv(X_test_final, None, processed_test_path)

Đã lưu file tại: ../data/processed/aug_train.csv
Đã lưu file tại: ../data/processed/aug_test.csv
